Chama os cachorros

In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

Pré-processamento

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|███████████████████████████████████████| 836/836 [00:04<00:00, 175.44it/s]


REDE CNN

In [28]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

#Definições
num_classes = 133 #Quantidade de raças de cachorros
input_shape = (224, 224, 3) #Tamnho das imagens de entrada

#Convolutional layer configuration
model = Sequential()

model.add(BatchNormalization(input_shape=input_shape))

model.add(Conv2D(filters=20, kernel_size=(7, 7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=20, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=20, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(GlobalAveragePooling2D())

#Dense layer
model.add(Dense(1024, activation='relu'))

#Dense layer 2
model.add(Dense(1024, activation='relu'))

#Output layer
model.add(Dense(num_classes, activation='softmax', name='output'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_74 (Batc (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 224, 224, 20)      560       
_________________________________________________________________
batch_normalization_75 (Batc (None, 224, 224, 20)      80        
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 224, 224, 20)      3620      
_________________________________________________________________
batch_normalization_76 (Batc (None, 224, 224, 20)      80        
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 224, 224, 20)      3620      
_________________________________________________________________
batch_normalization_77 (Batc (None, 224, 224, 20)      80        
__________

Compilar modelo

In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=20, batch_size=20, verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
4120/6680 [=================>............] - ETA: 1721s - loss: 4.9249 - acc: 0.0000e+ - ETA: 935s - loss: 4.8999 - acc: 0.0000e+00 - ETA: 673s - loss: 4.8986 - acc: 0.0000e+0 - ETA: 542s - loss: 4.8589 - acc: 0.0125    - ETA: 463s - loss: 4.8754 - acc: 0.010 - ETA: 410s - loss: 4.8245 - acc: 0.025 - ETA: 372s - loss: 4.8728 - acc: 0.021 - ETA: 343s - loss: 4.9096 - acc: 0.025 - ETA: 321s - loss: 4.9157 - acc: 0.022 - ETA: 303s - loss: 4.9526 - acc: 0.025 - ETA: 289s - loss: 4.9432 - acc: 0.031 - ETA: 276s - loss: 4.9682 - acc: 0.029 - ETA: 266s - loss: 4.9790 - acc: 0.026 - ETA: 257s - loss: 4.9671 - acc: 0.025 - ETA: 249s - loss: 4.9751 - acc: 0.023 - ETA: 242s - loss: 4.9758 - acc: 0.021 - ETA: 235s - loss: 4.9741 - acc: 0.020 - ETA: 230s - loss: 4.9763 - acc: 0.019 - ETA: 225s - loss: 4.9776 - acc: 0.018 - ETA: 220s - loss: 4.9748 - acc: 0.017 - ETA: 216s - loss: 4.9803 - acc: 0.019 - ETA: 212s - loss: 4.9736 - acc: 0.020 - 

4160/6680 [=================>............] - ETA: 166s - loss: 4.6604 - acc: 0.050 - ETA: 165s - loss: 4.5370 - acc: 0.050 - ETA: 164s - loss: 4.5097 - acc: 0.083 - ETA: 164s - loss: 4.4848 - acc: 0.087 - ETA: 163s - loss: 4.4495 - acc: 0.090 - ETA: 163s - loss: 4.4490 - acc: 0.083 - ETA: 162s - loss: 4.4692 - acc: 0.071 - ETA: 162s - loss: 4.4975 - acc: 0.062 - ETA: 161s - loss: 4.4711 - acc: 0.061 - ETA: 161s - loss: 4.4781 - acc: 0.055 - ETA: 160s - loss: 4.4716 - acc: 0.050 - ETA: 160s - loss: 4.4683 - acc: 0.054 - ETA: 159s - loss: 4.4709 - acc: 0.053 - ETA: 159s - loss: 4.4736 - acc: 0.050 - ETA: 158s - loss: 4.4538 - acc: 0.050 - ETA: 158s - loss: 4.4306 - acc: 0.053 - ETA: 157s - loss: 4.4347 - acc: 0.052 - ETA: 157s - loss: 4.4528 - acc: 0.052 - ETA: 156s - loss: 4.4599 - acc: 0.050 - ETA: 156s - loss: 4.4437 - acc: 0.047 - ETA: 155s - loss: 4.4273 - acc: 0.045 - ETA: 155s - loss: 4.4137 - acc: 0.045 - ETA: 154s - loss: 4.4131 - acc: 0.045 - ETA: 154s - loss: 4.4092 - acc: 0.0

6680/6680 [==============================] - ETA: 62s - loss: 4.3408 - acc: 0.04 - ETA: 61s - loss: 4.3407 - acc: 0.04 - ETA: 61s - loss: 4.3399 - acc: 0.04 - ETA: 60s - loss: 4.3416 - acc: 0.04 - ETA: 60s - loss: 4.3410 - acc: 0.04 - ETA: 59s - loss: 4.3420 - acc: 0.04 - ETA: 59s - loss: 4.3418 - acc: 0.04 - ETA: 58s - loss: 4.3409 - acc: 0.04 - ETA: 58s - loss: 4.3402 - acc: 0.04 - ETA: 57s - loss: 4.3402 - acc: 0.04 - ETA: 57s - loss: 4.3397 - acc: 0.04 - ETA: 56s - loss: 4.3401 - acc: 0.04 - ETA: 56s - loss: 4.3389 - acc: 0.04 - ETA: 55s - loss: 4.3404 - acc: 0.04 - ETA: 55s - loss: 4.3390 - acc: 0.04 - ETA: 54s - loss: 4.3382 - acc: 0.04 - ETA: 54s - loss: 4.3375 - acc: 0.04 - ETA: 53s - loss: 4.3357 - acc: 0.04 - ETA: 53s - loss: 4.3340 - acc: 0.05 - ETA: 52s - loss: 4.3332 - acc: 0.05 - ETA: 52s - loss: 4.3333 - acc: 0.05 - ETA: 51s - loss: 4.3334 - acc: 0.05 - ETA: 51s - loss: 4.3309 - acc: 0.05 - ETA: 50s - loss: 4.3294 - acc: 0.05 - ETA: 50s - loss: 4.3281 - acc: 0.05 - ETA: 

4160/6680 [=================>............] - ETA: 167s - loss: 3.4534 - acc: 0.150 - ETA: 167s - loss: 3.8067 - acc: 0.100 - ETA: 170s - loss: 3.8140 - acc: 0.083 - ETA: 169s - loss: 3.8304 - acc: 0.087 - ETA: 169s - loss: 3.8693 - acc: 0.100 - ETA: 168s - loss: 3.8981 - acc: 0.091 - ETA: 167s - loss: 3.8561 - acc: 0.100 - ETA: 167s - loss: 3.8634 - acc: 0.100 - ETA: 166s - loss: 3.8714 - acc: 0.088 - ETA: 165s - loss: 3.8335 - acc: 0.095 - ETA: 164s - loss: 3.8228 - acc: 0.090 - ETA: 164s - loss: 3.8369 - acc: 0.087 - ETA: 163s - loss: 3.8284 - acc: 0.088 - ETA: 162s - loss: 3.8207 - acc: 0.089 - ETA: 162s - loss: 3.8013 - acc: 0.093 - ETA: 161s - loss: 3.7992 - acc: 0.093 - ETA: 161s - loss: 3.7979 - acc: 0.097 - ETA: 160s - loss: 3.7873 - acc: 0.091 - ETA: 159s - loss: 3.7969 - acc: 0.086 - ETA: 159s - loss: 3.8148 - acc: 0.085 - ETA: 158s - loss: 3.8251 - acc: 0.083 - ETA: 158s - loss: 3.8112 - acc: 0.086 - ETA: 157s - loss: 3.8000 - acc: 0.087 - ETA: 157s - loss: 3.7978 - acc: 0.0

6680/6680 [==============================] - ETA: 63s - loss: 3.7839 - acc: 0.09 - ETA: 62s - loss: 3.7836 - acc: 0.09 - ETA: 62s - loss: 3.7850 - acc: 0.09 - ETA: 61s - loss: 3.7817 - acc: 0.09 - ETA: 61s - loss: 3.7826 - acc: 0.09 - ETA: 60s - loss: 3.7831 - acc: 0.09 - ETA: 60s - loss: 3.7828 - acc: 0.09 - ETA: 59s - loss: 3.7829 - acc: 0.09 - ETA: 59s - loss: 3.7849 - acc: 0.09 - ETA: 58s - loss: 3.7839 - acc: 0.09 - ETA: 58s - loss: 3.7849 - acc: 0.09 - ETA: 57s - loss: 3.7867 - acc: 0.09 - ETA: 57s - loss: 3.7861 - acc: 0.09 - ETA: 56s - loss: 3.7847 - acc: 0.09 - ETA: 56s - loss: 3.7817 - acc: 0.09 - ETA: 55s - loss: 3.7825 - acc: 0.09 - ETA: 55s - loss: 3.7800 - acc: 0.09 - ETA: 54s - loss: 3.7780 - acc: 0.09 - ETA: 54s - loss: 3.7782 - acc: 0.09 - ETA: 53s - loss: 3.7776 - acc: 0.09 - ETA: 52s - loss: 3.7756 - acc: 0.09 - ETA: 52s - loss: 3.7737 - acc: 0.09 - ETA: 51s - loss: 3.7738 - acc: 0.09 - ETA: 51s - loss: 3.7748 - acc: 0.09 - ETA: 50s - loss: 3.7736 - acc: 0.09 - ETA: 

...

6680/6680 [==============================] - ETA: 62s - loss: 0.6613 - acc: 0.79 - ETA: 61s - loss: 0.6604 - acc: 0.79 - ETA: 61s - loss: 0.6613 - acc: 0.79 - ETA: 60s - loss: 0.6630 - acc: 0.79 - ETA: 60s - loss: 0.6635 - acc: 0.79 - ETA: 59s - loss: 0.6653 - acc: 0.79 - ETA: 59s - loss: 0.6667 - acc: 0.79 - ETA: 58s - loss: 0.6666 - acc: 0.78 - ETA: 58s - loss: 0.6671 - acc: 0.78 - ETA: 57s - loss: 0.6659 - acc: 0.78 - ETA: 57s - loss: 0.6671 - acc: 0.78 - ETA: 56s - loss: 0.6677 - acc: 0.78 - ETA: 56s - loss: 0.6694 - acc: 0.78 - ETA: 55s - loss: 0.6703 - acc: 0.78 - ETA: 55s - loss: 0.6708 - acc: 0.78 - ETA: 54s - loss: 0.6719 - acc: 0.78 - ETA: 54s - loss: 0.6710 - acc: 0.78 - ETA: 53s - loss: 0.6726 - acc: 0.78 - ETA: 53s - loss: 0.6713 - acc: 0.78 - ETA: 52s - loss: 0.6706 - acc: 0.78 - ETA: 52s - loss: 0.6708 - acc: 0.78 - ETA: 51s - loss: 0.6744 - acc: 0.78 - ETA: 51s - loss: 0.6746 - acc: 0.78 - ETA: 50s - loss: 0.6748 - acc: 0.78 - ETA: 50s - loss: 0.6750 - acc: 0.78 - ETA: 

4160/6680 [=================>............] - ETA: 165s - loss: 1.2798 - acc: 0.700 - ETA: 165s - loss: 0.8332 - acc: 0.800 - ETA: 164s - loss: 0.6355 - acc: 0.833 - ETA: 164s - loss: 0.5565 - acc: 0.850 - ETA: 163s - loss: 0.5029 - acc: 0.860 - ETA: 163s - loss: 0.5268 - acc: 0.858 - ETA: 163s - loss: 0.4927 - acc: 0.864 - ETA: 162s - loss: 0.4641 - acc: 0.875 - ETA: 162s - loss: 0.4477 - acc: 0.877 - ETA: 161s - loss: 0.4885 - acc: 0.870 - ETA: 161s - loss: 0.4599 - acc: 0.877 - ETA: 160s - loss: 0.4649 - acc: 0.875 - ETA: 160s - loss: 0.4757 - acc: 0.869 - ETA: 159s - loss: 0.4555 - acc: 0.875 - ETA: 159s - loss: 0.4528 - acc: 0.880 - ETA: 158s - loss: 0.4794 - acc: 0.868 - ETA: 158s - loss: 0.4752 - acc: 0.867 - ETA: 157s - loss: 0.4626 - acc: 0.872 - ETA: 157s - loss: 0.4535 - acc: 0.871 - ETA: 156s - loss: 0.4661 - acc: 0.862 - ETA: 156s - loss: 0.4506 - acc: 0.869 - ETA: 155s - loss: 0.4488 - acc: 0.870 - ETA: 155s - loss: 0.4463 - acc: 0.869 - ETA: 154s - loss: 0.4475 - acc: 0.8

6680/6680 [==============================] - ETA: 62s - loss: 0.4586 - acc: 0.84 - ETA: 62s - loss: 0.4612 - acc: 0.84 - ETA: 61s - loss: 0.4627 - acc: 0.84 - ETA: 61s - loss: 0.4624 - acc: 0.84 - ETA: 60s - loss: 0.4629 - acc: 0.84 - ETA: 60s - loss: 0.4620 - acc: 0.84 - ETA: 59s - loss: 0.4615 - acc: 0.84 - ETA: 59s - loss: 0.4611 - acc: 0.84 - ETA: 58s - loss: 0.4625 - acc: 0.84 - ETA: 58s - loss: 0.4608 - acc: 0.84 - ETA: 57s - loss: 0.4606 - acc: 0.84 - ETA: 57s - loss: 0.4592 - acc: 0.84 - ETA: 56s - loss: 0.4594 - acc: 0.84 - ETA: 56s - loss: 0.4607 - acc: 0.84 - ETA: 55s - loss: 0.4596 - acc: 0.84 - ETA: 55s - loss: 0.4596 - acc: 0.85 - ETA: 54s - loss: 0.4593 - acc: 0.85 - ETA: 54s - loss: 0.4608 - acc: 0.85 - ETA: 53s - loss: 0.4596 - acc: 0.85 - ETA: 53s - loss: 0.4619 - acc: 0.85 - ETA: 52s - loss: 0.4615 - acc: 0.85 - ETA: 52s - loss: 0.4624 - acc: 0.85 - ETA: 51s - loss: 0.4631 - acc: 0.85 - ETA: 51s - loss: 0.4639 - acc: 0.85 - ETA: 50s - loss: 0.4648 - acc: 0.84 - ETA: 

4160/6680 [=================>............] - ETA: 168s - loss: 0.4027 - acc: 0.800 - ETA: 167s - loss: 0.2425 - acc: 0.900 - ETA: 167s - loss: 0.2573 - acc: 0.900 - ETA: 166s - loss: 0.2371 - acc: 0.912 - ETA: 166s - loss: 0.2203 - acc: 0.920 - ETA: 165s - loss: 0.2812 - acc: 0.908 - ETA: 165s - loss: 0.2646 - acc: 0.914 - ETA: 164s - loss: 0.3009 - acc: 0.900 - ETA: 164s - loss: 0.3050 - acc: 0.900 - ETA: 163s - loss: 0.3208 - acc: 0.895 - ETA: 163s - loss: 0.3015 - acc: 0.904 - ETA: 162s - loss: 0.2985 - acc: 0.904 - ETA: 162s - loss: 0.2983 - acc: 0.907 - ETA: 161s - loss: 0.2940 - acc: 0.910 - ETA: 161s - loss: 0.2983 - acc: 0.906 - ETA: 160s - loss: 0.2910 - acc: 0.909 - ETA: 160s - loss: 0.2942 - acc: 0.908 - ETA: 159s - loss: 0.2942 - acc: 0.908 - ETA: 159s - loss: 0.3023 - acc: 0.902 - ETA: 158s - loss: 0.3128 - acc: 0.897 - ETA: 158s - loss: 0.3135 - acc: 0.892 - ETA: 157s - loss: 0.3097 - acc: 0.895 - ETA: 157s - loss: 0.3221 - acc: 0.895 - ETA: 156s - loss: 0.3130 - acc: 0.9

6680/6680 [==============================] - ETA: 63s - loss: 0.3234 - acc: 0.89 - ETA: 63s - loss: 0.3241 - acc: 0.89 - ETA: 62s - loss: 0.3243 - acc: 0.89 - ETA: 62s - loss: 0.3245 - acc: 0.89 - ETA: 61s - loss: 0.3257 - acc: 0.89 - ETA: 61s - loss: 0.3247 - acc: 0.89 - ETA: 60s - loss: 0.3251 - acc: 0.89 - ETA: 60s - loss: 0.3269 - acc: 0.89 - ETA: 59s - loss: 0.3264 - acc: 0.89 - ETA: 58s - loss: 0.3264 - acc: 0.89 - ETA: 58s - loss: 0.3265 - acc: 0.89 - ETA: 57s - loss: 0.3270 - acc: 0.89 - ETA: 57s - loss: 0.3273 - acc: 0.89 - ETA: 56s - loss: 0.3284 - acc: 0.89 - ETA: 56s - loss: 0.3282 - acc: 0.89 - ETA: 55s - loss: 0.3283 - acc: 0.89 - ETA: 55s - loss: 0.3300 - acc: 0.89 - ETA: 54s - loss: 0.3293 - acc: 0.89 - ETA: 54s - loss: 0.3305 - acc: 0.89 - ETA: 53s - loss: 0.3303 - acc: 0.89 - ETA: 53s - loss: 0.3298 - acc: 0.89 - ETA: 52s - loss: 0.3292 - acc: 0.89 - ETA: 52s - loss: 0.3289 - acc: 0.89 - ETA: 51s - loss: 0.3299 - acc: 0.89 - ETA: 51s - loss: 0.3305 - acc: 0.89 - ETA: 

In [31]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 30.1435%


Salvar checkpoint do model.
Treinar novamente para melhorar.
Rodar varias vezes para ir melhorando.

In [35]:
from keras.callbacks import ModelCheckpoint  

model.save_weights('saved_models/weights.best.from_scratch.hdf5')

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [ ]:
model.fit(train_tensors, train_targets,
          validation_data=(valid_tensors, valid_targets),
          epochs=12, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/12
6680/6680 [==============================] - 165s - loss: 0.1570 - acc: 0.9497 - val_loss: 5.7356 - val_acc: 0.2754
Epoch 2/12


4160/6680 [=================>............] - ETA: 160s - loss: 0.2727 - acc: 0.850 - ETA: 161s - loss: 0.1443 - acc: 0.925 - ETA: 160s - loss: 0.1544 - acc: 0.916 - ETA: 161s - loss: 0.1270 - acc: 0.937 - ETA: 160s - loss: 0.1070 - acc: 0.950 - ETA: 160s - loss: 0.0925 - acc: 0.958 - ETA: 160s - loss: 0.1114 - acc: 0.950 - ETA: 160s - loss: 0.1157 - acc: 0.950 - ETA: 159s - loss: 0.1100 - acc: 0.950 - ETA: 159s - loss: 0.1131 - acc: 0.950 - ETA: 158s - loss: 0.1233 - acc: 0.945 - ETA: 158s - loss: 0.1137 - acc: 0.950 - ETA: 157s - loss: 0.1213 - acc: 0.950 - ETA: 157s - loss: 0.1199 - acc: 0.950 - ETA: 156s - loss: 0.1138 - acc: 0.953 - ETA: 156s - loss: 0.1104 - acc: 0.956 - ETA: 156s - loss: 0.1154 - acc: 0.955 - ETA: 155s - loss: 0.1126 - acc: 0.958 - ETA: 155s - loss: 0.1168 - acc: 0.957 - ETA: 154s - loss: 0.1147 - acc: 0.957 - ETA: 154s - loss: 0.1309 - acc: 0.952 - ETA: 153s - loss: 0.1273 - acc: 0.954 - ETA: 153s - loss: 0.1295 - acc: 0.954 - ETA: 152s - loss: 0.1307 - acc: 0.9

6680/6680 [==============================] - 173s - loss: 0.1604 - acc: 0.9507 - val_loss: 5.2915 - val_acc: 0.2802
Epoch 3/12
4160/6680 [=================>............] - ETA: 165s - loss: 0.5199 - acc: 0.850 - ETA: 165s - loss: 0.3044 - acc: 0.900 - ETA: 164s - loss: 0.2054 - acc: 0.933 - ETA: 164s - loss: 0.1852 - acc: 0.937 - ETA: 163s - loss: 0.1536 - acc: 0.950 - ETA: 163s - loss: 0.1396 - acc: 0.958 - ETA: 162s - loss: 0.1228 - acc: 0.964 - ETA: 162s - loss: 0.1125 - acc: 0.968 - ETA: 161s - loss: 0.1230 - acc: 0.961 - ETA: 161s - loss: 0.1185 - acc: 0.965 - ETA: 160s - loss: 0.1329 - acc: 0.954 - ETA: 160s - loss: 0.1231 - acc: 0.958 - ETA: 160s - loss: 0.1310 - acc: 0.957 - ETA: 159s - loss: 0.1290 - acc: 0.957 - ETA: 159s - loss: 0.1245 - acc: 0.960 - ETA: 158s - loss: 0.1176 - acc: 0.962 - ETA: 157s - loss: 0.1261 - acc: 0.958 - ETA: 157s - loss: 0.1206 - acc: 0.961 - ETA: 156s - loss: 0.1237 - acc: 0.957 - ETA: 156s - loss: 0.1397 - acc: 0.955 - ETA: 155s - loss: 0.1783 - a

In [ ]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

In [ ]:
Salvar checkpoint do model.
Treinar novamente para melhorar.
Rodar varias vezes para ir melhorando.

In [ ]:
from keras.callbacks import ModelCheckpoint  

model.save_weights('saved_models/weights.best.from_scratch.hdf5')

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [ ]:
model.fit(train_tensors, train_targets,
          validation_data=(valid_tensors, valid_targets),
          epochs=12, batch_size=20, callbacks=[checkpointer], verbose=1)

In [ ]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)